# First notebook for ALPS I Hands-on session


## Building circuits in `pytket`

### Building a circuit with the `Circuit` class


You can create a circuit by creating an instance of the `Circuit` class and adding gates manually.

#### Start with a simple one-qubit circuit and look at some of the basic gates seen in the lectures

* The `X` gate

In [2]:
from pytket import Circuit

circ = Circuit(1)
circ.X(0)

[X q[0]; ]

Now let's draw a nice picture of the circuit with the circuit renderer

In [3]:
from pytket.circuit.display import render_circuit_jupyter as draw

draw(circ)

The renderer has some useful interactive features including...
* Easy export to `.png`, `.jpeg` and `.svg` image formats
* Toggle quantum circuit/ZX calculus display
* adjustable zoom

Try them out!

Note that the state of each qubit starts from $|0\rangle$ by default

Each quantum circuit (without measurements) represents a unitary matrix. Let's have a look at the matrix for the circuit above

In [4]:
circ.get_unitary()

array([[0.+0.j, 1.+0.j],
       [1.+0.j, 0.+0.j]])

For small circuits we can also directly look at the statevector (the vector in the Hilbert space of the qubits in the circuit) that is reached at the end of the circuit

In [5]:
circ.get_statevector()

array([0.+0.j, 1.+0.j])

#### The `H` gate

*Exercise: write and inspect a circuit with a single Hadamard gate and compare the unitary matrix with what you have seen in the lecture.*

In [12]:
circ = Circuit(1)
circ.H(0)
draw(circ)
print(circ.get_unitary())
print(circ.get_statevector())

[[ 0.70710678+0.j  0.70710678+0.j]
 [ 0.70710678+0.j -0.70710678+0.j]]
[0.70710678+0.j 0.70710678+0.j]


#### Now move to a two-qubits circuit and prepare a Bell state

In [14]:
from pytket import Circuit

bell_circ = Circuit(2, name="Bell circuit")
bell_circ.H(0)
bell_circ.CX(0, 1)

[H q[0]; CX q[0], q[1]; ]

One can inspect basic elements of the circuits using attributes and methods of the `Circuit` class

In [26]:
bell_circ.get_commands()

[H q[0];, CX q[0], q[1];]

In [21]:
bell_circ.n_qubits

2

In [22]:
bell_circ.name

'Bell circuit'

In [24]:
bell_circ.n_1qb_gates()

1

In [25]:
bell_circ.n_2qb_gates()

1

We can draw the circuit as before

In [15]:
draw(bell_circ)

In [16]:
bell_circ.get_unitary()

array([[ 0.70710678+0.j,  0.        +0.j,  0.70710678+0.j,
         0.        +0.j],
       [ 0.        +0.j,  0.70710678+0.j,  0.        +0.j,
         0.70710678+0.j],
       [ 0.        +0.j,  0.70710678+0.j,  0.        +0.j,
        -0.70710678+0.j],
       [ 0.70710678+0.j,  0.        +0.j, -0.70710678+0.j,
         0.        +0.j]])

In [17]:
bell_circ.get_statevector()

array([0.70710678+0.j, 0.        +0.j, 0.        +0.j, 0.70710678+0.j])

We can also get some pretty latex expressions for the unitary and statevector representations, including symbolic values!

In [19]:
from pytket.utils.symbolic import circuit_to_symbolic_unitary

circuit_to_symbolic_unitary(bell_circ)

Matrix([
[1/sqrt(2),         0,  1/sqrt(2),          0],
[        0, 1/sqrt(2),          0,  1/sqrt(2)],
[        0, 1/sqrt(2),          0, -sqrt(2)/2],
[1/sqrt(2),         0, -sqrt(2)/2,          0]])

In [20]:
from pytket.utils.symbolic import circuit_apply_symbolic_statevector

circuit_apply_symbolic_statevector(bell_circ)

Matrix([
[sqrt(2)/2],
[        0],
[        0],
[sqrt(2)/2]])

### A controlled unitary rotation 

Here we will use symbols in a gate!

In [29]:
from sympy import symbols

angles = symbols(["β", "γ", "δ"])

circ = Circuit(1, name="Unitary rotation")
circ.U3(angles[0], angles[1], angles[2], 0)

[U3(β, γ, δ) q[0]; ]

In [30]:
draw(circ)

In [31]:
circuit_to_symbolic_unitary(circ)

Matrix([
[            cos(pi*β/2),      -exp(I*pi*δ)*sin(pi*β/2)],
[exp(I*pi*γ)*sin(pi*β/2), exp(I*pi*(γ + δ))*cos(pi*β/2)]])

To make this a controlled unitary on a second qubit, we can create a circuit with 2 qubits and add a controlled operation

In [48]:
cu_circ = Circuit(2, name="Controlled rotation")
cu_circ.CU3(angles[0], angles[1], angles[2], 0, 1)
draw(cu_circ)

In [49]:
circuit_to_symbolic_unitary(cu_circ)

Matrix([
[1, 0,                       0,                                   0],
[0, 1,                       0,                                   0],
[0, 0,             cos(pi*β/2),            -exp(I*pi*δ)*sin(pi*β/2)],
[0, 0, exp(I*pi*γ)*sin(pi*β/2), exp(I*pi*γ)*exp(I*pi*δ)*cos(pi*β/2)]])

#### A 3-qubit GHZ state: including measurements that are needed when running the circuit on devices

In [7]:
from pytket import Circuit

ghz_circ = Circuit(3)
ghz_circ.H(0)
ghz_circ.CX(0, 1)
ghz_circ.CX(1, 2)
ghz_circ.add_barrier(ghz_circ.qubits)
ghz_circ.measure_all()

[H q[0]; CX q[0], q[1]; CX q[1], q[2]; Barrier q[0], q[1], q[2]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; Measure q[2] --> c[2]; ]

#### Adding more complicated gates

Some operations aren't directly available through the `Circuit` class and need to be added using the `OpType` enum: [Full List Here](https://tket.quantinuum.com/api-docs/optype.html#pytket.circuit.OpType)

In [36]:
from pytket import OpType

cnry_circ = Circuit(4)
cnry_circ.add_gate(OpType.CnRy, angles[0], [0, 1, 2, 3]) # add a multi-controlled Ry gate, angles first followed by qubits
draw(cnry_circ)

In [38]:
# circuit_to_symbolic_unitary(cnry_circ) does not work on this specific gate, since it has not been implemented.
# we will see later how to decompose complicated gates into simpler ones

See also the [Circuit construction](https://tket.quantinuum.com/user-manual/manual_circuit.html) section of the user manual.

### Build a `Circuit` from a QASM file

We can import a circuit from a QASM file using [pytket.qasm](https://tket.quantinuum.com/api-docs/qasm.html). There are also functions for generating a circuit from a QASM string or exporting to a qasm file.


In [44]:
from pytket.qasm import circuit_from_qasm

w_state_circ = circuit_from_qasm("W-state.qasm")
draw(w_state_circ)

There are also analogous functions in [pytket.qasm](https://tket.quantinuum.com/api-docs/qasm.html) for exporting a pytket `Circuit` to QASM format.

### Import a circuit from qiskit (or other SDK)

Its possible to generate a `pytket` circuit directly from a qiskit `QuantumCircuit` using the [qiskit_to_tk](https://tket.quantinuum.com/extensions/pytket-qiskit/api.html#pytket.extensions.qiskit.tk_to_qiskit) function.

In [45]:
from qiskit import QuantumCircuit

qiskit_circ = QuantumCircuit(3)
qiskit_circ.h(range(3))
qiskit_circ.ccx(2, 1 ,0)
qiskit_circ.cx(0, 1)
print(qiskit_circ)

     ┌───┐┌───┐     
q_0: ┤ H ├┤ X ├──■──
     ├───┤└─┬─┘┌─┴─┐
q_1: ┤ H ├──■──┤ X ├
     ├───┤  │  └───┘
q_2: ┤ H ├──■───────
     └───┘          


Interoperability with `qiskit` is coming from a `pytket` extension called `pytket-qiskit`.

In [46]:
from pytket.extensions.qiskit import qiskit_to_tk

tket_circ = qiskit_to_tk(qiskit_circ)

draw(tket_circ)

In [47]:
from pytket.utils import gate_counts

print(dict(gate_counts(tket_circ)))

{<OpType.H: 33>: 3, <OpType.CCX: 61>: 1, <OpType.CX: 45>: 1}


*Note that pytket and qiskit use opposite qubit ordering conventions. So circuits which look identical may correspond to different unitary operations.*



Circuit conversion functions are also available for [pytket-cirq](https://tket.quantinuum.com/extensions/pytket-cirq/), [pytket-pennylane](https://tket.quantinuum.com/extensions/pytket-pennylane/), [pytket-braket](https://tket.quantinuum.com/extensions/pytket-braket/) and more.

## Useful circuit primitives

Here are a list of circuit subroutines that you might find useful when building circuits

* [CircBox](https://tket.quantinuum.com/api-docs/circuit.html#pytket.circuit.CircBox) - Allows you define higher level subroutines using a `Circuit` which can then be reused in complex circuits and algorithms.

* [QControlBox](https://tket.quantinuum.com/api-docs/circuit.html#pytket.circuit.QControlBox) - Construct custom controlled gates in pytket using any base unitary operation. See the manual section on [controlled gates](https://tket.quantinuum.com/user-manual/manual_circuit.html#controlled-box-operations)

* [Unitary2qBox](https://tket.quantinuum.com/api-docs/circuit.html#pytket.circuit.Unitary2qBox) - Allows you to generate an `Circuit` to implement an arbitrary 2 qubit operation. Unitary synthesis also available for 1 and 3 qubit operations. See the [manual section](https://tket.quantinuum.com/user-manual/manual_circuit.html#boxes-for-unitary-synthesis) for usage.

* [StatePreparationBox](https://tket.quantinuum.com/api-docs/circuit.html#pytket.circuit.StatePreparationBox) - Allows preparation of an arbitary quantum state using a naive algorithm. See [the docs](https://tket.quantinuum.com/user-manual/manual_circuit.html#multiplexors-state-preperation-boxes-and-toffolibox) for more. 



#### Try out the `Unitary1qBox` and `Unitary2qBox`

In [39]:
from pytket.circuit import Circuit, Unitary1qBox, Unitary2qBox
import numpy as np

# encode a unitary matrix defined as an array into your circuit: 2x2 matrix -> single qubit
u1 = np.asarray([[2 / 3, (-2 + 1j) / 3], [(2 + 1j) / 3, 2 / 3]])
u1box = Unitary1qBox(u1)

# 4x4 matrix -> two qubits
u2 = np.asarray([[0, 1, 0, 0], [0, 0, 0, -1], [1, 0, 0, 0], [0, 0, -1j, 0]])
u2box = Unitary2qBox(u2)

`pytket` lets you add _boxes_ to your circuit in a compositional way

In [40]:
circ = Circuit(3, name="Unitary boxes")
circ.add_unitary1qbox(u1box, 0)
circ.add_unitary2qbox(u2box, 1, 2)
circ.add_unitary1qbox(u1box, 2)
circ.add_unitary2qbox(u2box, 1, 0)

[Unitary1qBox q[0]; Unitary2qBox q[1], q[2]; Unitary2qBox q[1], q[0]; Unitary1qBox q[2]; ]

In [41]:
draw(circ)


## Other useful resources on Circuit construction

* [Circuit construction section of the user manual](https://tket.quantinuum.com/user-manual/manual_circuit.html)

* Example notebook on [Quantum Phase Estimation](https://tket.quantinuum.com/examples/phase_estimation.html) Show's the use of several circuit primitives to build circuits for quantum phase estimation



## Basic circuit rewrites


### Rebases (converting a circuit to a desired gateset)

Before you can run a circuit on a quantum device you have to convert the circuit to the gateset supported by the device. In pytket this conversion is called a rebase

Here's a basic example converting the `Controlled rotation` circuit to a set of simple gates




In [50]:
from pytket.passes.auto_rebase import auto_rebase_pass

simple_rebase = auto_rebase_pass({OpType.X, OpType.SX, OpType.Rz, OpType.CX})

simple_rebase.apply(cu_circ)

draw(cu_circ)

Here the conversion can be done directly. This isn't always the case.

For more on rebase passes see 
* manual section on [rebases](https://tket.quantinuum.com/user-manual/manual_compiler.html#rebases)
* This [stack exchange post](https://quantumcomputing.stackexchange.com/questions/31504/how-to-specify-custom-gate-set-for-the-optimization-in-pytket/31505#31505) 








### Peephole Optimisation






Peephole optimsations are a general purpose type of optimisations that are designed to reduce the two qubit gate count. This is done by performing local rewrites to the circuit including re-Synthesising optimal subcircuits. The resulting circuit contains only one and two qubit gates.

For more on circuit optimisation read the compilation section of the pytket user manual

In [51]:
from pytket.passes import FullPeepholeOptimise, DecomposeBoxes

DecomposeBoxes().apply(cu_circ)  # remove boxes prior to optimising the circuit

FullPeepholeOptimise().apply(cu_circ)

draw(cu_circ)

For more on circuit optimisation read the [compilation section](https://tket.quantinuum.com/user-manual/manual_compiler.html) of the pytket user manual.